In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Считаем данные
colic = pd.read_csv("../input/horse-colic/horse.csv")
colic

In [ ]:
# Посчитаем кол-во отсутствующих данных
null = colic.isnull().sum() 
# Переведем в %
null = null / len(colic)*100
null = null[null>0]
null.sort_values(inplace=True, ascending=False)

null = null.to_frame()
null.columns = ['count']

from matplotlib import pyplot as plt

# Отобразим на графике
plt.figure(figsize=(20, 10))
sb.set(font_scale=1.2)
plt.style.use('ggplot')
sb.barplot(x=null.index, y='count', data=null)
plt.show()

In [ ]:
# Заменим наличие операции и возраст
colic = colic.replace({'no':0, 'yes': 1, 'adult':1, 'young':2})
colic

In [ ]:
# Заполним отсутствующие поля
colic.rectal_temp = colic.rectal_temp.fillna(value=colic.rectal_temp.mode()[0])
colic.pulse = colic.pulse.fillna(value=colic.pulse.mean())
colic.respiratory_rate = colic.respiratory_rate.fillna(value=colic.respiratory_rate.mean())
colic.abdomo_protein = colic.abdomo_protein.fillna(value=colic.abdomo_protein.mode()[0])
colic.total_protein = colic.total_protein.fillna(value=colic.total_protein.mean())
colic.packed_cell_volume = colic.packed_cell_volume.fillna(value=colic.packed_cell_volume.mean())
colic.nasogastric_reflux_ph = colic.nasogastric_reflux_ph.fillna(value=colic.nasogastric_reflux_ph.mean())

col = null.index
for i in col:
    colic[i] = colic[i].fillna(colic[i].mode()[0])

In [ ]:
# Проверим остались ли незаполненные поля
plt.figure(figsize=(12, 8))
sb.heatmap(colic.isnull())
plt.show()

In [ ]:
# Переведем строки в числа
from sklearn.preprocessing import LabelEncoder
col = colic.columns

for i in col:
    lb = LabelEncoder()
    lb.fit(colic[i].values)
    colic[i] = lb.transform(colic[i].values)

colic

In [ ]:
colic.outcome.value_counts()

In [ ]:
# Изобразим текущие результаты
plt.figure(figsize=(8, 5))
sb.countplot(x='outcome', data=colic)
plt.show()

In [ ]:
# Разделим данные
X = colic.drop('outcome', axis=1).values
y = colic['outcome'].values

In [ ]:
# Разделим данные на тренировочные и тестовые
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [ ]:
# Используем доступные алгоритмы типизации
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

algo = {'LR':LogisticRegression(),
        'DT':DecisionTreeClassifier(),
        'RFC':RandomForestClassifier(n_estimators=100),
        'SVM':SVC(gamma=0.001),
        'KNN':KNeighborsClassifier(n_neighbors=10)}

In [ ]:
# Результат
for k, v in algo.items():
    model = v
    model.fit(X_train, y_train)
    print('Acurracy of ' + k + ' is {0:.2f}'.format(model.score(X_test, y_test)*100))

In [ ]:
# На основне получненых данных создадим модель, которая будет решать задачу
model = RandomForestClassifier(n_estimators=100)
model.fit(X_test, y_test)
y_pred = model.predict(X_test)

In [ ]:
for x in range(0, 4):
    print("Prediction value " + str(model.predict([X_test[x]])))
    print("Real value " + str(y_test[x]) + "\n")